## Window Functions
---

A window function performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function.
But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. 
Behind the scenes, the window function is able to access more than just the current row of the query result.

Example:
To get avergae salary of a department along with the emoloyees' salaries:
(To aggregate measures (continous variables) over categories from categorical variables)

In [ ]:
SELECT depname
        ,empno
        ,salary
        ,avg(salary) OVER (PARTITION BY depname) 
FROM empsalary;

  depname  | empno | salary |          avg          
-----------+-------+--------+-----------------------
 develop   |    11 |   5200 | 5020.0000000000000000
 develop   |     7 |   4200 | 5020.0000000000000000
 develop   |     9 |   4500 | 5020.0000000000000000
 develop   |     8 |   6000 | 5020.0000000000000000
 develop   |    10 |   5200 | 5020.0000000000000000
 personnel |     5 |   3500 | 3700.0000000000000000
 personnel |     2 |   3900 | 3700.0000000000000000
 sales     |     3 |   4800 | 4866.6666666666666667
 sales     |     1 |   5000 | 4866.6666666666666667
 sales     |     4 |   4800 | 4866.6666666666666667
(10 rows)


The fourth column represents an average taken across all the table rows that have the same depname value as the current row.
(This actually is the same function as the regular avg aggregate function, 
but the `OVER` clause causes it to be treated as a window function and computed across an appropriate set of rows.) 

A window function call **always contains an `OVER` clause** directly following the window function's name and argument(s). This is what syntactically distinguishes it from a regular function or aggregate function. The OVER clause determines exactly how the rows of the query are split up for processing by the window function. **The `PARTITION BY` list within OVER specifies dividing the rows into groups, or partitions, that share the same values of the PARTITION BY expression(s).** For each row, the window function is computed across the rows that fall into the same partition as the current row.

---
You can also control the order in which rows are processed by window functions using ORDER BY within OVER. (The window ORDER BY does not even have to match the order in which the rows are output.) Here is an example:

In [ ]:
SELECT depname
        ,empno
        ,salary
        ,rank() OVER (PARTITION BY depname ORDER BY salary DESC)
        FROM empsalary;

  depname  | empno | salary | rank 
-----------+-------+--------+------
 develop   |     8 |   6000 |    1
 develop   |    10 |   5200 |    2
 develop   |    11 |   5200 |    2
 develop   |     9 |   4500 |    4
 develop   |     7 |   4200 |    5
 personnel |     2 |   3900 |    1
 personnel |     5 |   3500 |    2
 sales     |     1 |   5000 |    1
 sales     |     4 |   4800 |    2
 sales     |     3 |   4800 |    2
(10 rows)

As shown here, the rank function produces a numerical rank within the current row's partition for each distinct ORDER BY value, in the order defined by the ORDER BY clause. rank needs no explicit parameter, because its behavior is entirely determined by the OVER clause.

**`The rows considered by a window function are those of the "virtual table" produced by the query's FROM clause as filtered by its WHERE, GROUP BY, and HAVING clauses if any. For example, a row removed because it does not meet the WHERE condition is not seen by any window function. A query can contain multiple window functions that slice up the data in different ways by means of different OVER clauses, but they all act on the same collection of rows defined by this virtual table.`**

We already saw that ORDER BY can be omitted if the ordering of rows is not important. **It is also possible to omit PARTITION BY, in which case there is just one partition containing all the rows.**

---

There is another important concept associated with window functions: for each row, there is a set of rows within its partition called its `window frame`. Many (but not all) window functions act only on the rows of the window frame, rather than of the whole partition. **By default, if ORDER BY is supplied then the frame consists of all rows from the start of the partition up through the current row, plus any following rows that are equal to the current row according to the ORDER BY clause. When ORDER BY is omitted the default frame consists of all rows in the partition.**

Here is an example using sum:

In [ ]:
SELECT salary
        ,sum(salary) OVER ()
         FROM empsalary;

 salary |  sum  
--------+-------
   5200 | 47100
   5000 | 47100
   3500 | 47100
   4800 | 47100
   3900 | 47100
   4200 | 47100
   4500 | 47100
   4800 | 47100
   6000 | 47100
   5200 | 47100
(10 rows)

Above, since there is no ORDER BY in the OVER clause, the window frame is the same as the partition, which for lack of PARTITION BY is the whole table; in other words each sum is taken over the whole table and so we get the same result for each output row. But if we add an ORDER BY clause, we get very different results:

In [ ]:
SELECT salary
        ,sum(salary) OVER (ORDER BY salary)
        FROM empsalary;

 salary |  sum  
--------+-------
   3500 |  3500
   3900 |  7400
   4200 | 11600
   4500 | 16100
   4800 | 25700
   4800 | 25700
   5000 | 30700
   5200 | 41100
   5200 | 41100
   6000 | 47100
(10 rows)

Here the sum is taken from the first (lowest) salary up through the current one, including any duplicates of the current one (notice the results for the duplicated salaries).

**`Window functions are permitted only in the SELECT list and the ORDER BY clause of the query.`** They are forbidden elsewhere, such as in GROUP BY, HAVING and WHERE clauses. This is because they logically execute after the processing of those clauses. **`Also, window functions execute after regular aggregate functions. This means it is valid to include an aggregate function call in the arguments of a window function, but not vice versa.`**

---

If there is a need to filter or group rows after the window calculations are performed, you can use a sub-select. For example:

In [ ]:
SELECT depname, empno, salary, enroll_date
FROM
  (SELECT depname, empno, salary, enroll_date,
          rank() OVER (PARTITION BY depname ORDER BY salary DESC, empno) AS pos
     FROM empsalary
  ) AS ss
WHERE pos < 3;

The above query only shows the rows from the inner query having rank less than 3.

In [ ]:
SELECT sum(salary) OVER w
            ,avg(salary) OVER w
    FROM empsalary
    WINDOW w AS (PARTITION BY depname ORDER BY salary DESC);

Link: https://www.postgresql.org/docs/9.1/tutorial-window.html

### ROW_NUMBER()

**`ROW_NUMBER()`** does just what it sounds like—displays the number of a given row. It starts are 1 and numbers the rows according to the ORDER BY part of the window statement. ROW_NUMBER() does not require you to specify a variable within the parentheses:

In [ ]:
SELECT start_terminal,
       start_time,
       duration_seconds,
       ROW_NUMBER() OVER (ORDER BY start_time)
                    AS row_number
  FROM tutorial.dc_bikeshare_q1_2012
 WHERE start_time < '2012-01-08'

start_terminal|     start_time    |duration_seconds|row_number
         31245|2012-01-01 00:04:00|475             | 1
         31400|2012-01-01 00:10:00|1162            | 2
         31400|2012-01-01 00:10:00|1145            | 3
         31101|2012-01-01 00:15:00|485             | 4
         31102|2012-01-01 00:15:00|471             | 5
         31017|2012-01-01 00:17:00|358             | 6
         31236|2012-01-01 00:18:00|1754            | 7
         31101|2012-01-01 00:22:00|259             | 8
         31014|2012-01-01 00:24:00|516             | 9
         31101|2012-01-01 00:25:00|913             | 10

Using the `PARTITION BY` clause will allow you to begin counting 1 again in each partition. The following query starts the count over again for each terminal:

In [ ]:
SELECT start_terminal,
       start_time,
       duration_seconds,
       ROW_NUMBER() OVER (PARTITION BY start_terminal
                          ORDER BY start_time)
                    AS row_number
  FROM tutorial.dc_bikeshare_q1_2012
 WHERE start_time < '2012-01-08'

start_terminal	start_time	duration_seconds	row_number
31000	2012-01-01 15:32:00	74	1
31000	2012-01-02 12:40:00	291	2
31000	2012-01-02 19:15:00	520	3
31000	2012-01-03 07:22:00	424	4
31000	2012-01-03 07:22:00	447	5
31001	2012-01-03 12:32:00	1422	1
31002	2012-01-04 17:36:00	348	2
31003	2012-01-05 15:13:00	277	3
31004	2012-01-05 17:25:00	3340	4
31005	2012-01-06 07:28:00	414	5

### RANK() and DENSE_RANK()

**`RANK()`** is slightly different from `ROW_NUMBER()`. If you order by `start_time`, for example, it might be the case that some terminals have rides with two identical start times. In this case, they are given the same rank, whereas ROW_NUMBER() gives them different numbers. In the following query, you notice the 4th and 5th observations for start_terminal 31000—they are both given a rank of 4, and the following result receives a rank of 6:

In [ ]:
SELECT start_terminal,
       duration_seconds,
       RANK() OVER (PARTITION BY start_terminal
                    ORDER BY start_time)
              AS rank
  FROM tutorial.dc_bikeshare_q1_2012
 WHERE start_time < '2012-01-08'

start_terminal	duration_seconds	rank
31000	74	1
31000	291	2
31000	520	3
31000	424	4
31000	447	4
31000	1422	6
31000	348	7
31000	277	8
31000	3340	9
31000	414	10

You can also use **`DENSE_RANK()`** instead of RANK() depending on your application. Imagine a situation in which three entries have the same value. Using either command, they will all get the same rank. For the sake of this example, let's say it's "2." Here's how the two commands would evaluate the next results differently:

* RANK() would give the identical rows a rank of 2, then skip ranks 3 and 4, so the next result would be 5
* DENSE_RANK() would still give all the identical rows a rank of 2, but the following row would be 3—no ranks would be skipped.


### NTILE

You can use window functions to identify what percentile (or quartile, or any other subdivision) a given row falls into. The syntax is **`NTILE(*# of buckets*)`**. In this case, `ORDER BY` determines which column to use to determine the quartiles (or whatever number of 'tiles you specify). For example:

In [ ]:
SELECT start_terminal,
       duration_seconds,
       NTILE(4) OVER
         (PARTITION BY start_terminal ORDER BY duration_seconds)
          AS quartile,
       NTILE(5) OVER
         (PARTITION BY start_terminal ORDER BY duration_seconds)
         AS quintile,
       NTILE(100) OVER
         (PARTITION BY start_terminal ORDER BY duration_seconds)
         AS percentile
  FROM tutorial.dc_bikeshare_q1_2012
 WHERE start_time < '2012-01-08'
 ORDER BY start_terminal, duration_seconds

### LAG and LEAD

It can often be useful to compare rows to preceding or following rows, especially if you've got the data in an order that makes sense. You can use `LAG` or `LEAD` to create columns that pull values from other rows—all you need to do is enter which column to pull from and how many rows away you'd like to do the pull. LAG pulls from previous rows and LEAD pulls from following rows:

In [ ]:
SELECT start_terminal,
       duration_seconds,
       LAG(duration_seconds, 1) OVER
         (PARTITION BY start_terminal ORDER BY duration_seconds) AS lag,
       LEAD(duration_seconds, 1) OVER
         (PARTITION BY start_terminal ORDER BY duration_seconds) AS lead
  FROM tutorial.dc_bikeshare_q1_2012
 WHERE start_time < '2012-01-08'
 ORDER BY start_terminal, duration_seconds

start_terminal	duration_seconds	lag	lead
31000	74		277
31000	277	74	291
31000	291	277	348
31000	348	291	387
31000	387	348	393
31000	393	387	398
31000	398	393	399
31000	399	398	412
31000	412	399	414
31000	414	412	424

This is especially useful if you want to calculate differences between rows:

In [ ]:
SELECT start_terminal,
       duration_seconds,
       duration_seconds -LAG(duration_seconds, 1) OVER
         (PARTITION BY start_terminal ORDER BY duration_seconds)
         AS difference
  FROM tutorial.dc_bikeshare_q1_2012
 WHERE start_time < '2012-01-08'
 ORDER BY start_terminal, duration_seconds

start_terminal	duration_seconds	difference
31000	74	
31000	277	203
31000	291	14
31000	348	57
31000	387	39
31000	393	6
31000	398	5
31000	399	1
31000	412	13
31000	414	2

The first row of the `difference` column is null because there is no previous row from which to pull. Similarly, using `LEAD` will create nulls at the end of the dataset. If you'd like to make the results a bit cleaner, you can wrap it in an outer query to remove nulls:

In [ ]:
SELECT *
  FROM (
    SELECT start_terminal,
           duration_seconds,
           duration_seconds -LAG(duration_seconds, 1) OVER
             (PARTITION BY start_terminal ORDER BY duration_seconds)
             AS difference
      FROM tutorial.dc_bikeshare_q1_2012
     WHERE start_time < '2012-01-08'
     ORDER BY start_terminal, duration_seconds
       ) sub
 WHERE sub.difference IS NOT NULL

start_terminal	duration_seconds	difference
31000	277	203
31000	291	14
31000	348	57
31000	387	39
31000	393	6
31000	398	5
31000	399	1
31000	412	13
31000	414	2
31000	424	10

Source: https://mode.com/sql-tutorial/sql-window-functions/